In [20]:
import extruct
import requests
from pprint import pprint
from bs4 import BeautifulSoup
from w3lib.html import get_base_url
from recipe_scrapers import scrape_me

In [2]:
urls = ['https://cookieandkate.com/best-tahini-sauce-recipe/',
        'https://www.bonappetit.com/recipe/chicken-tikka-masala',
        'https://www.delish.com/cooking/recipe-ideas/recipes/a46330/skillet-sicilian-chicken-recipe/'
       ]

In [3]:
def get_html(url: str, headers: dict={}) -> bytes:
    headers = {
        'Access-Control-Allow-Origin': '*',
        'Access-Control-Allow-Methods': 'GET',
        'Access-Control-Allow-Headers': 'Content-Type',
        'Access-Control-Max-Age': '3600',
        'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0'
    }
    response = requests.get(url, headers=headers)
    return response.text

In [4]:
def get_metadata(html: bytes, url: str) -> dict:
    metadata = extruct.extract(
        html,
        base_url=get_base_url(url),
        syntaxes=['json-ld'],
        uniform=True
    )['json-ld']
    if bool(metadata) and isinstance(metadata, list):
        metadata = metadata[0]
    return metadata

In [5]:
def scrape(url: str, pretty_print: bool=False) -> dict:
    """Parse structured data from a target page."""
    html = get_html(url)
    metadata = get_metadata(html, url)
    if pretty_print:
        pprint(metadata, indent=2, width=150)
    return metadata

In [7]:
metadata = [scrape(url) for url in urls]

In [11]:
metadata[0].keys()

dict_keys(['@context', '@graph'])

In [12]:
metadata[1].keys()

dict_keys(['@context', '@type', 'articleBody', 'alternativeHeadline', 'keywords', 'thumbnailUrl', 'publisher', 'isPartOf', 'isAccessibleForFree', 'author', 'aggregateRating', 'description', 'image', 'headline', 'name', 'recipeIngredient', 'recipeInstructions', 'recipeYield', 'url', 'dateModified', 'datePublished'])

In [13]:
metadata[2].keys()

dict_keys(['@graph', '@context', 'url', 'publisher', '@type', 'author', 'datePublished', 'headline', 'image', 'mainEntityOfPage', 'thumbnailUrl', 'dateModified', 'isAccessibleForFree', 'hasPart', 'name', 'prepTime', 'cookTime', 'totalTime', 'recipeIngredient', 'recipeInstructions', 'video', 'recipeCuisine', 'aggregateRating', 'review', 'recipeCategory', 'recipeYield', 'description', 'keywords'])

In [14]:
import re
def cleanhtml(raw_html):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', raw_html)
    return cleantext

In [18]:
metadata[2]['recipeIngredient']

['1 tbsp. <p>extra-virgin olive oil</p>',
 '6 <p>bone-in, skin-on chicken thighs (about 2 pounds)</p>',
 '<p>Kosher salt</p>',
 '<p>Freshly ground black pepper</p>',
 '2 <p>cloves garlic, minced</p>',
 '1 tbsp. <p>fresh thyme leaves</p>',
 '1 tsp. <p>crushed red pepper flakes</p>',
 '3/4 c. <p>low-sodium chicken broth</p>',
 '1/2 c. <p>heavy cream</p>',
 '1/2 c. <p>chopped sun-dried tomatoes</p>',
 '1/4 c. <p>freshly grated Parmesan</p>',
 '<p>Freshly torn basil, for serving</p>']

- Remove `<p>` tags from ingredient